In [ ]:
import os
from ctc_src import  *
os.chdir("/home/hemant/E2E_NER-Through-Speech/S2T/")
from opts import add_decoder_args, add_inference_args
from utils import load_model

import numpy as np
import torch
from tqdm.auto import tqdm
from data.data_loader import SpectrogramParser

from opts import add_decoder_args, add_inference_args
from utils import load_model

In [ ]:
labels = "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ "
alpha, beta, lm, oov_weight =1.96, 4, '/home/hemant/4_gram.arpa', 1
scorer = Scorer(alpha,beta,lm,oov_weight)

prune = 0.0001
beam_size = 50

In [ ]:
def read_csv(d):
    with open(d,"r") as f:
        return f.readlines()
    
torch.set_grad_enabled(False)
device = torch.device("cuda")
model = load_model(device, "/home/hemant/asr_wm/models/deep/final.pth")
spect_parser = SpectrogramParser(model.audio_conf, normalize=True)

torch.cuda.set_device(int(0))

csv = read_csv('/home/hemant/asr_wm/data/ner/align_wav_txt/dev.csv')

In [7]:
total_wer, num_tokens = 0, 0

for i in tqdm(csv[:1]):
    audio_path, reference_path = i.split(",")

    spect = spect_parser.parse_audio(audio_path).contiguous()
    spect = spect.view(1, 1, spect.size(0), spect.size(1))
    spect = spect.to(device)

    input_sizes = torch.IntTensor([spect.size(3)]).int()
    out, output_sizes = model(spect, input_sizes)
    out = out.cpu().detach().numpy()[0]

    transcript = ctc_best_path(out,labels)
    
    with open(reference_path.replace("\n",""),"r") as f:
        reference = f.readline()
    
    wer_inst = wer_(transcript,reference)
    total_wer += wer_inst
    num_tokens += len(reference.split(' '))
        
wer = (float(total_wer) / num_tokens)*100
print('Test Summary \t'
    'Average WER {wer:.3f}\t'.format(wer=wer))


Test Summary 	Average WER 23.077	


In [ ]:
alpha, beta = np.linspace(0.1,4,25), np.linspace(0.1,25,25)
values = [[i,j] for j in beta for i in alpha]

total_wer, num_tokens = 0, 0
output = 'alpha\tbeta\twer\n'

print(values[-1])
for alpha,beta in tqdm(values):
    scorer.reset_params(alpha,beta)
    print("aplha: ",alpha,"beta: ",beta)
    
    for i in csv[:100]:
        audio_path, reference_path = i.split(",")

        spect = spect_parser.parse_audio(audio_path).contiguous()
        spect = spect.view(1, 1, spect.size(0), spect.size(1))
        spect = spect.to(device)

        input_sizes = torch.IntTensor([spect.size(3)]).int()
        out, output_sizes = model(spect, input_sizes)
        out = out.cpu().detach().numpy()[0]

        transcript = prefix_bsp(out,labels,scorer,log=False,prune=prune,beam_size=beam_size)
    
        with open(reference_path.replace("\n",""),"r") as f:
            reference = f.readline()

        wer_inst = wer_(transcript,reference)
        total_wer += wer_inst
        num_tokens += len(reference.split(' '))

    wer = (float(total_wer) / num_tokens)*100
    print('Test Summary \t'
            'Average WER {wer:.3f}\t'.format(wer=wer))
    
    output +=f'{alpha}\t{beta}\t{wer}\n'
    with open("/home/hemant/ctc_decoders/abw.json", "w") as f:
        f.write(output)

In [4]:
import multiprocessing

In [10]:
def prefix_bsbatch(out,labels,scorer,log=False,prune=0.00001,beam_size=25,w_t_o_b=50,num_processes=8):
    if not num_processes > 0:
        raise ValueError("Number of processes must be positive!")

    # use global variable to pass the externnal scorer to beam search decoder
    global ext_nproc_scorer
    ext_nproc_scorer = scorer
    nproc = True

    pool = multiprocessing.Pool(processes=num_processes)
    results = []
    for i, probs_list in enumerate(out):
        args = (out,labels,scorer,log,prune,beam_size,50,
                nproc)
        results.append(pool.apply_async(prefix_bsp, args))

    pool.close()
    pool.join()
    beam_search_results = [result.get() for result in results]
    return beam_search_results

In [11]:
prefix_bsbatch(out,labels,scorer,log=False,prune=prune,beam_size=beam_size)

NameError: name '_kenlm' is not defined